# Introduction to Apache Beam API

This notebook contains interactive exercises which introduces to [**Apache Beam**](https://beam.apache.org/) API which are needed for doing Differentially Private (`DP`) computations. 

In this notebook, we will first familiarise with Apache Beam and its [Python SDK](https://beam.apache.org/documentation/sdks/python/) by walking through multiple exercises of increasing complexity. 

**Don't worry**: if you are not yet familiar with Apache Beam and its Python APIs, each exercise will include a _Hints_ subsection that will point you in the right direction, suggesting what you should be looking at in the documentation to solve the exercise. Then, if you are still struggling with the exercise - or simply to compare your results - you can look at the attached _Solution_.

In the next notebook, we will use Apache Beam to implement a fully _differentially private_ data analysis pipeline.

## Instructions

We recommend opening and running this notebook in **Google Colab** by simply clicking on the badge below.

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PipelineDP/blob/main/docs/tutorial_1/1_beam_introduction.ipynb)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/OpenMined/PipelineDP/blob/main/docs/tutorial_1/1_beam_introduction.ipynb)

[![GitHub](https://img.shields.io/badge/view%20on-GitHub-blue)](https://github.com/OpenMined/PipelineDP/blob/main/docs/tutorial_1/1_beam_introduction.ipynb)

Once in Colab, here is what you should do: 

1. Click "Run in Google Colab" link above
2. Make a copy in Drive (File $\mapsto$ Save a copy in Drive)
3. Run all the cells in **Setup** area
4. Enjoy the exercises :)

In [ ]:
#@title Setup: Install and import `apache-beam` 
from IPython.display import clear_output
try:
    import apache_beam as beam
except ImportError:
    !pip install "apache-beam[interactive]"    
    clear_output()
finally:
    import apache_beam as beam
    print("apache_beam package available and imported correctly ✅")
    print(f"Apache Beam Python SDK version: {beam.__version__}")

## Dataset

In this notebook we will be using the [**Netflix Prize Dataset**](https://en.wikipedia.org/wiki/Netflix_Prize) as a driver example.

> The Netflix Prize was an open competition for the best collaborative filtering algorithm to predict user ratings for films, based on previous ratings without any other information about the users or films, i.e. without the users or the films being identified except by numbers assigned for the contest.

(_Source_: Wikipedia)

The choice of this dataset is particularly interesting from our perspective of _differentially private_ data pipelines, since this dataset was used as the reference example to present a new class of statistical de-anonymisation attacks against high-dimensional data. 
More details on these techniques, and their application to the `Netflix Prize` dataset are available in this [article](https://www.schneier.com/blog/archives/2007/12/anonymity_and_t_2.html), and in the original paper $\Rightarrow$ [**Robust De-anonymization of Large Sparse Datasets**](https://www.cs.utexas.edu/~shmat/shmat_oak08netflix.pdf).

### The `Netflix-Prize` Dataset

The Netflix Prize dataset (`Netflix-prize`) is available on [Kaggle](https://www.kaggle.com/netflix-inc/netflix-prize-data). 

We will setup the Python [`kaggle`](https://pypi.org/project/kaggle/) official API to automatically download and use the dataset.

**(A)** Let's first setup the `DATASET_FOLDER` where the dataset will be downloaded and saved:

In [ ]:
#@title Run to set up Data and Output folders { display-mode: "form" }
import os
from pathlib import Path


NETFLIX_PRIZE_FOLDER = Path("netflix_prize_dataset")
DATASET_FOLDER = NETFLIX_PRIZE_FOLDER / "data"
OUTPUT_FOLDER = NETFLIX_PRIZE_FOLDER  / "outputs"

DATA_FILE = str(DATASET_FOLDER / "movie_views.txt")
OUTFILE_TEMPLATE = str(OUTPUT_FOLDER / '{}.txt')

os.makedirs(DATASET_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

**(B)** Download the `Netflix-prize` dataset from Kaggle

In [ ]:
#@title Run to setup `kaggle` Python API { display-mode: "form" }
try:
    import kaggle
except ImportError:
    !pip install kaggle
    clear_output()
except OSError:
    print("Kaggle package is available, but credentials needs to be setup.")
else:
    print("kaggle package imported correctly ✅")

**Create and place your API token**

In order to download datasets using the Kaggle API, please create and download your API token from Kaggle website:

1. Log in to [Kaggle](https://www.kaggle.com/) website.
2. Go to your **Account** (locate your profile avatar in the top-right corner of the page)

![kaggle-account](images/kaggle_account.png)

3. Scroll down your account page until you find the **API cell**.
4. Click on to "Create new API token" button.

![kaggle-api-key](images/api_key.png)

This will download a file called `kaggle.json`. This file contains the credentials required to use the Kaggle APIs. 

5. Open this file, and include this data into the form below:

In [ ]:
#@title Setup `kaggle` API credentials { display-mode: "form" }
from ipywidgets import Password
from ipywidgets import widgets
from IPython.display import display
from IPython.display import Image
import os
import stat
import json

kaggle_api_credentials = os.path.expanduser('~/.kaggle/kaggle.json')

API_CREDENTIALS_EXIST = os.path.exists(kaggle_api_credentials) 

if not API_CREDENTIALS_EXIST:
    user = widgets.Text(
        placeholder='Insert your Kaggle User Name!',
        description='User Name',
        disabled=False
    )

    pwd = Password(
        description='Key',
        placeholder='Insert key for user {}'.format(user.value)
    )

    def store_kaggle_credentials(wdgt):
        folder = os.path.expanduser('~/.kaggle')
        os.makedirs(folder, exist_ok=True)
        file = '{}/kaggle.json'.format(folder)
        with open(file, 'w') as kaggle_file:
            json.dump({
                'username': user.value,
                'key': wdgt.value
            }, kaggle_file)

            print("Credentials created in {} ✅".format(folder))
        os.chmod(file, stat.S_IREAD | stat.S_IWRITE)

    user.on_submit(lambda wdg: display(pwd))
    pwd.on_submit(store_kaggle_credentials)

    display(user)
else:
    print("Kaggle API credentials found ✅ - All set, Ready to start!")

In [ ]:
#@title Authenticate on Kaggle via APIs { display-mode: "form" }
import kaggle

kaggle.api.authenticate()

In [ ]:
#@title **Search** on Kaggle for available Netflix-Prize datasets { display-mode: "form" }
!kaggle datasets list -s "netflix-prize"

The top on the list is the dataset we are looking for: `netflix-inc/netflix-prize-data`.

The dimension of the archive is `638 MB`, which means it may take some time to download and uncompress. (⚠️ **this may take sometime, depending on your internet connection**) Please keep this in mind while you will run the next cell.

In [ ]:
#@title Run this cell to initiate the download { display-mode: "form" }
if not len(os.listdir(DATASET_FOLDER)) or not os.path.exists(DATA_FILE):
    # Checking here to avoid download multiple times in case of notebook re-run
    kaggle.api.dataset_download_files("netflix-inc/netflix-prize-data", 
                                      path=DATASET_FOLDER, unzip=True, quiet=False)

In [ ]:
#@title **Inspect** the content of the Data folder { display-mode: "form" }
os.listdir(DATASET_FOLDER)

#### `Netflix-prize` data files

Before getting into the actual processing of this dataset with **Apache Beam**, let's briefly describe the actual data files included in the Kaggle archive. 

The dataset is organised into multiple files, accounting for over `100` million ratings from `480` thousand
randomly-chosen, anonymous Netflix customers, over `17` thousand movie titles. 

- The `probe.txt` and the `qualitfying.txt` files are needed for the submission on Kaggle, so discarded as not interesting here.

- Movie information (i.e. _metadata_) are contained in the `movie_titles.txt` in the following format:

```
MovieID,YearOfRelease,Title

- MovieID is a general (randomised) ID for movies, rangin from 1 to 17,770.
- YearOfRelease can range from 1,890 to 2,005.
- Title (in English) is the Netflix title.
```

- The various `combined_data_*.txt` contain the actual data in the following format:

```
MovieID:
CustomerID,Rating,Date
CustomerID,Rating,Date
....

- MovieIDs matches those in the list of "movie_titles.txt".
- CustomerIDs range from 1 to 2,649,429, with gaps. There are 480,189 users.
- Ratings are on a five star (integral) scale from 1 to 5.
- Dates have the format YYYY-MM-DD.
```

A more thorough description of each of those files is available in the [`README`](netflix_prize_dataset/data/README) file. 

#### Preparing our dataset 

The size of the whole dataset ($\approxeq$ `2GB` if joining together all the `combined_data_` files) doesn't allow to process the entire collection as quickly and interactive as we would expect into an interactive environment like Jupyter notebooks. This is still true despite all the boosting trick we will be setting up on `beam` later, nonetheless. 

For this reason, we will limit our analysis to a reduced version of the dataset. In the next two cells, we will quickly gather some statistics on the size of each data file (in terms of `number of lines`), and we will interactively choose how many of these lines we will retain in our working version.

**Note**: Without any loss of generality, we will use `combined_data_1.txt` as for the reference data file. As you will soon realise, it won't make much difference. However, feel free to experiment by changing the reference file, or even by joining all of them together, if you feel your computing environment is able to handle this dataset in a reasonable amount of time (_more on this later_).

In [ ]:
#@title Let's see how many lines each data file contains { display-mode: "form" }
data_files = filter(lambda f: f.startswith("combined_data_"), os.listdir(DATASET_FOLDER))
lines_count = 0
for data_file in sorted(data_files):
    file_path = DATASET_FOLDER / data_file
    wc_l = !wc -l $file_path
    lines, _ = wc_l[0].strip().split()
    lines_count += int(lines)
    print(f"{data_file}:  {int(lines):,}")
print(f"Total Lines Count  : {lines_count:,}")

From these statistics, we can immediately notice that each data file roughly contains the same number of lines, which means that the amout of time required to process each one of them will be more or less the same.

However, even just limiting our analysis to one single file (i.e. `>20M` lines) might be prohibitive, and indeed beyond of the scope of this tutorial. 

The **main goal** of the tutorial is to demostrate the ability of `apache_beam` to scale up the computation on large datasets. For real scalability however, we might also need decent computation capacity (_and we generally don't, if running this notebook on our laptop_).

Therefore, let's decide how many lines we want our _reduced_ dataset to keep! 

After a few testing on the performance, we will be retaining just `500K` lines by **default** as this will result in a _few seconds_ execution on Google Colab (on the default VM instances with just _2 cores_ ).

➡️ Feel free to increase this baseline if you are up to process a much larger dataset. For example, when running this notebook on my laptop (`mac OS` with `8 cores`) I was able to get `10 secs` execution time for `1M` lines.

In [ ]:
#@title Setup the size (in lines) of the (_reduced_) Netflix-prize Dataset {display-mode: "form"}

#@markdown Different sizes (i.e. `#lines`) will generally scale up linearly with the computation time.

# Note: this piece of code is a bit weird when read into a notebook, but it makes much
# more sense if run into Google Colab as it uses the new Form feature
DATASET_SIZE = "500K lines" #@param ["10K lines", "100K lines", "500K lines", "1M lines", "5M lines", "10M lines", "15M lines", "20M lines", "Full Dataset"] {type: "string"}

if DATASET_SIZE == "Full Dataset":
    DS_LINES = 24058263  # whole combined_data_1.txt file
else:
    DS_LINES = int(DATASET_SIZE.replace(" lines", "").replace("K", "000").replace("M", "000000"))

print("Selected Number of Lines: {:,}".format(DS_LINES))

In [ ]:
#@title **Create** the reference data file `movie_views.txt` { display-mode: "form" }

!head -n $DS_LINES $DATASET_FOLDER/combined_data_1.txt > $DATA_FILE 
!du -h $DATA_FILE

⚠️ **WARNING**

While preparing this notebook, I realised that the `movie_titles.csv` file is not originally encoded as `utf8`, but the `latin-1` which is unfortunately **not supported** in `apache_beam`.

Therefore, we are going to convert file encoding to be compatible with `apache_beam` for later use.

In [ ]:
!cp $DATASET_FOLDER/movie_titles.csv $DATASET_FOLDER/movie_titles_latin1.csv

In [ ]:
with open(DATASET_FOLDER/"movie_titles_latin1.csv", encoding="latin-1") as mtl, \
open(DATASET_FOLDER/"movie_titles.csv", "wb") as mtut8:
    ll = mtl.read()
    mtut8.write(ll.encode("utf-8"))
    

✅ Well done! 

The setup of the `Netflix-prize` dataset is now complete! Now it is time to start setting up our execution framework based on **Apache Beam**.

---

## The Execution Framework

In this section we will define the core main components that will be used throughout the exercises. These components will be based on **Apache Beam**, which consitutes the reference computational framework, and provides the building blocks to define our data workflows.  

### `apache_beam` in a Nutshell

Apache Beam adheres to a programming model which is centred around a few key elements:

* `Pipeline`: determines the **main** _data processing workflow_ as defined as a _sequence_ of **operations** on the data. This sequence is internally encoded as a directed acyclic graph in which nodes and arcs correspond to instances of `PCollection` and `PTransform`, respectively. 


- `PCollection`: represents a generic (**parallel**, `P`) data collection that can be either _bounded_ or _unbounded_ in size.


- `PTransform`: encodes a single (transform) operation. Each single `PTransform` accepts a `PCollection` in input, and it is expected to return a new `PCollection`. Beam provides some [**core**](https://beam.apache.org/documentation/programming-guide/#core-beam-transforms) transformers (e.g. [`ParDo`](https://beam.apache.org/documentation/programming-guide/#pardo), [`Flatten`](https://beam.apache.org/documentation/programming-guide/#flatten), and [`Partition`](https://beam.apache.org/documentation/programming-guide/#partition)) which correspondo to specific processing paradigms.


- `Runner`: (or `PipelineRunner`) determines the computational environment in which the whole data processing will be executed. 

Finally, pipelines I/O operations are supported by the `apache_beam` framework as core primitives (i.e. `PTransform`) to `Read` and `Write` from and to external sources.

$\Rightarrow$ More information on the Basics of [Beam Model](https://beam.apache.org/documentation/basics/) and on [Pipeline I/O](https://beam.apache.org/documentation/programming-guide/#pipeline-io) are available in the online documentation.

### Core Execution Modules on `Netflix-Prize` dataset

Despite of its simplicity, the reference programming model in Apache Beam accounts for one very important feature: **composability**. In fact, it is possible to either re-use (part of) a `Pipeline` within other and more complex workflows, or simply chaining multiple pipelines together. 

With specific reference to the `Netflix-prize` dataset, the general default workflow is composed by the following steps:

1. **[S1]** Read from the `DATA_FILE`, and generate the reference `PCollection`
    - `S1.1` A proper **data abstraction** is necessary to encapsulate the information for each _movie view_;
    - `S1.2` Data I/O will be performed in a parallel fashion (i.e. `ParDo`), leveraging on one of the most important feature of Apache Beam: _Data Parallelization_. 


2. **[S2]** Pass on the generated `PCollection` to the core (_abstract_) processing module 
    - `S2.1` Each core module will be expected as a _callable_ (implemented in the following exercises) to return the final `PCollection` to store and save (see `S3`).

3. **[S3]** Write the results to the corresponding `OUTPUT_FILE`.

#### **`S1.1`** Data Abstraction

To encapsulate the information for each _movie view_, we will make use of Python 3.7+ `dataclass` (see [doc](https://docs.python.org/3.8/library/dataclasses.html)). 

Our reference `MovieView` dataclass model is very simple, and will represent the atomic **data object** included in the `netflix-prize` `PCollection`. 

To simplify objects instantiation, the `date_stamp` init attribute (i.e. [`InitVar`](https://docs.python.org/3.8/library/dataclasses.html#init-only-variables)), will be later converted with proper data type and format in the `date` field.

In [ ]:
from dataclasses import dataclass, InitVar
from datetime import datetime

@dataclass
class MovieView:
    user_id: int
    movie_id: int
    rating: int
    date_stamp: InitVar[str]
    date: datetime.date = None
    
    def __post_init__(self, date_stamp):
        if self.date is None and date_stamp is not None:
            self.date = datetime.strptime(date_stamp, '%Y-%m-%d')

Similarly, the `MovieTitle` data class will encapsulate the `metadata` information extracted per single movie (see `movie_titles.txt` data file:

In [ ]:
@dataclass
class MovieTitle:
    movie_id: int
    year: int
    name: str

##### Python Type Safety and Custom objects

The Apache Beam SDK for Python uses type hints during pipeline construction and runtime to try to emulate the correctness guarantees achieved by true static typing. Additionally, using type hints lays some groundwork that allows the backend service to perform efficient type deduction and registration of `Coder` objects.

Since in this tutorial we want to handle our own custom **data object**, we also need to implement our custom (_yet simple_) `beam.coders.Coder` to allow for a correct object _serialisation_.

**Note**: This step is particularly crucial during the execution (_in parallel_) of the Pipeline, to avoid `apache_beam` to fallback to the default `Serializer` (i.e. `Coder`) based in `pickle`.
More information on **Python Type Safety** in Apache Beam pipelines is available in the official [documentation](https://beam.apache.org/documentation/sdks/python-type-safety/).

In [ ]:
class MovieViewCoder(beam.coders.Coder):
    """Beam Coder Serialiser for custom MovieView Data Object"""
    def encode(self, view):
        return (f"{view.user_id}:{view.movie_id}:{view.rating}:{str(view.date.date())}").encode("utf-8")

    def decode(self, s):
        return MovieView(*s.decode("utf-8").split(':'))

    def is_deterministic(self):
        return True
    

class MovieTitleCoder(beam.coders.Coder):
    """Beam Coder Serialiser for custom MovieTitle Data Object"""
    def encode(self, title):
        return (f"{title.movie_id}--{title.year}--{title.name}").encode("utf-8")
    
    def decode(self, s):
        return MovieTitle(*s.decode("utf-8").split('--'))
    
    def is_deterministic(self):
        return True


# Once defined, we can register the new Coders
beam.coders.registry.register_coder(MovieView, MovieViewCoder)
beam.coders.registry.register_coder(MovieTitle, MovieTitleCoder)

##### Workaround `multiprocessing` issues with namespace

⚠️ **Note**: As a workaround to enable _parallel computation_ in the notebook via Python `multiprocessing` and `beam.DoFn` (see next section), the dataclass and its coder will be saved into an external module (i.e. `movie_view.py`) for later use via **direct import**. This will guarantee a proper object serialisation, while also adhering to **Apache Beam** requirements. See [Requirements for writing user code for beam transforms](https://beam.apache.org/documentation/programming-guide/#requirements-for-writing-user-code-for-beam-transforms) for more information.

In [ ]:
%%bash 

echo -e "
from dataclasses import dataclass, InitVar
from datetime import datetime
import apache_beam as beam


@dataclass
class MovieView:
    user_id: int
    movie_id: int
    rating: int
    date_stamp: InitVar[str]
    date: datetime.date = None
    
    def __post_init__(self, date_stamp):
        if self.date is None and date_stamp is not None:
            self.date = datetime.strptime(date_stamp, '%Y-%m-%d')
            

@dataclass
class MovieTitle:
    movie_id: int
    year: int
    name: str
            

class MovieViewCoder(beam.coders.Coder):
    \"\"\"Beam Coder Serialiser for our custom MovieView Data Object\"\"\"
    def encode(self, view):
        return (f'{view.user_id}:{view.movie_id}:{view.rating}:{str(view.date.date())}').encode('utf-8')

    def decode(self, s):
        return MovieView(*s.decode('utf-8').split(':'))

    def is_deterministic(self):
        return True

class MovieTitleCoder(beam.coders.Coder):
    \"\"\"Beam Coder Serialiser for custom MovieTitle Data Object\"\"\"
    def encode(self, title):
        return (f'{title.movie_id}:{title.year}:{title.name}').encode('utf-8')
    
    def decode(self, s):
        return MovieTitle(*s.decode('utf-8').split(':'))
    
    def is_deterministic(self):
        return True


# Once defined, we can register the new Coders
beam.coders.registry.register_coder(MovieView, MovieViewCoder)
beam.coders.registry.register_coder(MovieTitle, MovieTitleCoder)
" > netflix_movies.py

In [ ]:
#@title **Verify** that import from movie_view module works properly { display-mode: "form" } 
try: 
    from netflix_movies import MovieView, MovieTitle
except ImportError:
    print("netflix_movies module cannot be imported. Please check!")
else:
    print("MovieView and MovieTitle dataclasses have been successfully imported.")

#### **`S1.2`** `beam.ParDo` handler

The [`beam.ParDo`](https://beam.apache.org/releases/pydoc/2.28.0/apache_beam.transforms.core.html#apache_beam.transforms.core.ParDo) processing paradigm is very similar to the `Map` phase of a `Map`/`Shuffle`/`Reduce`-style algorithm. 

The [`ParDo`](https://beam.apache.org/documentation/transforms/python/elementwise/pardo/) transformer is executed in parallel elementwise to the `PCollection`, via custom [`beam.DoFn`](https://beam.apache.org/releases/pydoc/2.28.0/apache_beam.transforms.core.html#apache_beam.transforms.core.DoFn) callable.

In [ ]:
@beam.typehints.with_output_types(MovieView)  # type-hint annotation for the output
class ParseMovieViews(beam.DoFn):
    """ParDo main execution core. Each line of the DATA_FILE will be processed, 
    and an instance of MovieView will be created and returned, accordingly."""
    
    def __init__(self):
        self.movie_id: int = -1
    
    def process(self, line: str):
        from netflix_movies import MovieView
        line = line.strip()  # get rid of any useless tabulation
        if not line:
            return  # Skip Line
        if line.endswith(":"):
            movie_id, _ = line.split(":")
            self.movie_id = int(movie_id)
            return  # Automatically skip this line
        user_id, rating, date_stamp, *_ = line.split(',')
        user_id, rating = int(user_id), int(rating)
        yield MovieView(user_id, self.movie_id, rating, date_stamp)

The `ParseMovieViews` callable will be used to generate the initial reference `PCollection` of `MovieView` instances for the `Netflix-prize` dataset.

We can now work out a solution in a very similar fashion to parse _movie_titles_ as extracted from the `movie_titles.txt` dataset: 

In [ ]:
@beam.typehints.with_output_types(MovieTitle)  # type-hint annotation for the output
class ParseMovieTitles(beam.DoFn):
    """ParDo main execution core. Each line of the DATA_FILE will be processed, 
    and an instance of MovieTitle will be created and returned, accordingly."""
    
    def process(self, line: str):
        from netflix_movies import MovieTitle
        line = line.strip()  # get rid of any useless tabulation
        if not line:
            return  # Skip Line
        movie_id, year, *name = line.split(',')  # title may contain a comma as well
        movie_id, name = int(movie_id), ",".join(name)
        year = -1 if year == "NULL" else int(year)
        yield MovieTitle(movie_id, year, name)

#### **`S2.1`** The core `beam.Pipeline`

Now it is finally the time to create the whole `beam.Pipeline` infrastructure, that will allow pluggable callables to be used (as implemented in the following Exercises).
The Pipeline will be also configured to leverage as much as possible parallel execution in a way that we could circumvent the Python [**GIL**](https://wiki.python.org/moin/GlobalInterpreterLock).

**Generating PCollections**

The first step will be to create a function that returns the reference `PCollection` we want to deal with. Having two separate **data objects** (i.e. `MovieView` and `MovieTitle`), we might assume we will be needing two separate functions to gather instances of either type.

In [ ]:
def netflix_movie_views_collection(p: beam.Pipeline, data_file: str = DATA_FILE) -> beam.PCollection[MovieView]:
    """Generate the initial (Parallel) Collection of 
    `MovieView` objects as extracted from the input `data_file`
    
    Parameters
    ----------
    p: beam.Pipeline
        The current reference pipeline object to run
    data_file: str (default: DATASET_FOLDER/movie_views.txt)
        Path (as str, as expected by `apache_beam`) to the reference dataset file
        
    Returns
    -------
    beam.PCollection: The (parallel) collection of `MovieView` instances, ready for further processing.
    """
    return (
        p
        | "Read MovieViews from Data File" >> beam.io.ReadFromText(data_file)
        | "Parse into MovieView objects" >> beam.ParDo(ParseMovieViews()).with_output_types(MovieView)
    )

In [ ]:
def netflix_movie_title_collection(p: beam.Pipeline, data_file: str = str(DATASET_FOLDER/"movie_titles.csv")) -> beam.PCollection[MovieTitle]:
    """Generate the initial (Parallel) Collection of 
    `MovieView` objects as extracted from the input `data_file`
    
    Parameters
    ----------
    p: beam.Pipeline
        The current reference pipeline object to run
    data_file: str (default: DATASET_FOLDER/movie_views.txt)
        Path (as str, as expected by `apache_beam`) to the reference dataset file
        
    Returns
    -------
    beam.PCollection: The (parallel) collection of `MovieTitle` instances, ready for further processing.
    """
    return (
        p
        | "Read MOvieTitle from Data File" >> beam.io.ReadFromText(data_file)
        | "Parse into MovieTitle objects" >> beam.ParDo(ParseMovieTitles()).with_output_types(MovieTitle)
    )

**Run Pipeline**

The last component of our framework will be devoted the actual _execution_ of the pipeline. In particular, we will define the _core_ `run_pipeline` function which will accept the following three parameters:

- `pipeline_fn`: the _pluggable_ `PipelineModule` to execute;


- `pcollection_fn`: the selected callable `PCollectionFun` to call to gather the target data objects `PCollection` (either `netflix_movie_views_collection` or `netflix_movie_title_collection`)


- `running_mode`: execution mode to be passed on as `PipelineOption`. Accepted values are (I) `multi_processing` (default); (II) `multi_threading`; (III) `in_memory`. More information on the pipeline options are available in the official [documentation](https://beam.apache.org/documentation/patterns/pipeline-options/);



- `data_file`: string matching the path to the input `data_file` (to be passed to the `netflix_movie_views_collection` function.

In [ ]:
# A PipelineModule is defined as a Callable that accepts any parameter
# and returns a beam.PCollection` instance

from typing import Callable

PipelineModule = Callable[[beam.PCollection], beam.PCollection]
PCollectionFun = Callable[[beam.Pipeline, str], beam.PCollection]

In [ ]:
#@title **Check** Python multiprocessing settings { display-mode: "form" }
import multiprocessing as mp

print("Python Multiprocessing Start method: ", mp.get_start_method())
print("Multiprocessing available Cores: ", mp.cpu_count())

In [ ]:
import time
from apache_beam.runners import DirectRunner
from apache_beam.options.pipeline_options import PipelineOptions

def run_pipeline(pipeline_fn: PipelineModule, 
                 pcollection_fn: PCollectionFun = netflix_movie_views_collection,
                 running_mode: str = "multi_processing", 
                 data_file: str = DATA_FILE):
    start_time = time.time()
    fn_name = pipeline_fn.__name__
    outfile = str(OUTFILE_TEMPLATE).format(fn_name)  # output logfile

    # Pipeline Options
    pipeline_options = PipelineOptions.from_dictionary(
        {
            "direct_num_workers": 0,  # setting parallelism to CPU count - alternatively: mp.cpu_count() - 1
            "direct_running_mode": running_mode,  # work around GIL
            "job-server-timeout": 65536,  # used to avoid potential timeout interruptions of workers
        }
    )
    with beam.Pipeline(runner=DirectRunner(), options=pipeline_options) as p:
        views_coll = pcollection_fn(p, data_file=data_file)
        pipeline_fn(views_coll) | "Write result" >> beam.io.WriteToText(
            outfile, shard_name_template=""
        )
    # Timing
    elapsed_time = time.time() - start_time
    hours, remainder = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Elapsed time in {fn_name} is {hours:2.0f}H {minutes:2.0f}m {seconds:2.0f}s")

⚠️ A few notes on the `PipelineOptions` used:

- `running_mode` is set to `multi_processing` by default as to work around the Python GIL, and so leverage on a full parallelization capacity in Python environment. However, when running on **Google Colab**, this won't make much of a difference as per execution time since only `2 cores` are available. Therefore, either `in_memory` or `multi_processing` will have more or less the same execution time. 


- `job-server-timeout` is set to `65536` to avoid any potential timeout in the backend that may happen. Please feel free to increase this number considerably when running on larger dataset. (See [`grpc:16038`](https://github.com/grpc/grpc/issues/16038))

---

## Exercises

Each exercise it to write an Apache Beam pipeline, which does some transformations on a part of Netflix prize dataset data. 

The completion of the exercises requires **additional research** on usage of Beam API with a search engine or looking at the offcial [API reference](https://beam.apache.org/documentation/sdks/python/). 

Let's start by looking at first lines of the dataset. 

In [ ]:
!head -n 10 $DATA_FILE

##### RECAP

The exercise framework function **`run_pipeline`** executes a user defined pipeline function.

The output of the pipeline is written in a file `outputs/(pipeline_fn name)`.


**Note**:
1. Each exercise has a solution, to test and verify your own implementation; 

2. In all exercises elements of the input collection are `MovieView` objects.



Let's start with an example: 

**A**) Write a pipeline function for computing number of records in the dataset. 
The example code is in the following code cell:

In [ ]:
from apache_beam.transforms import combiners

In [ ]:
# Number of records in the dataset

# The exercise framework calls this function with |views| representing all MovieViews in the dataset.
# |views| has type PCollection, which is a parrallel collection.
# Beam pipelines consist of applying PTransforms (i.e. parallel transformations) on PCollections
def count_all_views(views: beam.PCollection): 
   # This is an example of applying PTransform
   # Beam uses | (pipe) syntax:
   # new_p_collection = p_collection | "Optional comment" >> PTransform
   return views | "Count all views" >> combiners.Count.Globally().with_input_types(MovieView)


run_pipeline(count_all_views)  

print("\nFirst lines of the output file:")
!head -1 $OUTPUT_FOLDER/"count_all_views.txt"

## Exercise 1

Write a pipeline function for computing **distinct** movies IDs in the dataset

In [ ]:
def distinct_movies_ids(views):
    # TODO: write your code here
    pass

run_pipeline(distinct_movies_ids)

print("\Content of the output file:")
!cat $OUTPUT_FOLDER/"distinct_movies_ids.txt"

In [ ]:
#@title Hint (double click to open)

#@markdown Search for `beam.Map` and `beam.Distinct`

In [ ]:
#@title Solution (click on the arrow button to run, double click to open)
def distinct_movies_ids(views):
    return (
        views 
        | beam.Map(lambda mv: mv.movie_id)
        | beam.Distinct())

run_pipeline(distinct_movies_ids)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"distinct_movies_ids.txt"

## Exercise 2

Write a pipeline function for computing the **number** of distinct movies ids in the dataset

In [ ]:
def number_distinct_movies_ids(views):
    # TODO: write your code here
    
run_pipeline(number_distinct_movies_ids)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"number_distinct_movies_ids.txt"

In [ ]:
#@title Hint

# You could join the functions from the previous two exercises

In [ ]:
#@title Solution (click on the arrow button to run, double click to open)
def number_distinct_movies_ids(views):
    return (
        distinct_movies_ids(views) 
        | combiners.Count.Globally())

run_pipeline(number_distinct_movies_ids)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"number_distinct_movies_ids.txt"

## Exercise 3

Write a pipeline which computes the number of views per movie 
i.e. it outputs `PCollection(movie_id, number of views)`.

In [ ]:
def views_count_per_movie(views):
    # TODO: write your code

run_pipeline(views_count_per_movie)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"views_count_per_movie.txt"

In [ ]:
#@title Hint (double click to open)
#@markdown Search for `beam.combiners.Count.PerElement` or beam.combiners.Count.PerKey`

In [ ]:
#@title Solution
def views_count_per_movie(views):
    return ( 
        views 
        | beam.Map(lambda mv: mv.movie_id)
        | "Count movies" >> combiners.Count.PerElement())

run_pipeline(views_count_per_movie)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"views_count_per_movie.txt"

## Exercise 4

Write a pipeline which computes the _average_ rating per movie, i.e. it outputs `PCollection(movie_id, average rating)`.

In [ ]:
def mean_rating_per_movie(views):
    # TODO: write your code here

run_pipeline(mean_rating_per_movie)

print("\nContent of the output file:")
!cat "outputs/mean_rating_per_movie.txt"

In [ ]:
#@title Solution

def mean_rating_per_movie(views):
    # 2 element tuples then tuples are treated as (key, value) in PCollections
    return (
        views 
        | beam.Map(lambda mv: (mv.movie_id, mv.rating))
        | combiners.Mean.PerKey())

run_pipeline(mean_rating_per_movie)

print("\nContent of the output file:")
!cat $OUTPUT_FOLDER/"mean_rating_per_movie.txt"

## Exercise 5

Write a pipeline which bounds number of views per user by `|max_views|`, it outputs `PCollection(MovieView)`, such that for each `user_id` there are not more than `|max_views|` records.

In [ ]:
def bound_number_of_views_per_user(views, max_views=2):
    # TODO: write your code here

run_pipeline(bound_number_of_views_per_user)

print("\nContent of the output file:")
!head -5 $OUTPUT_FOLDER/"bound_number_of_views_per_user.txt"

print("\nNumber of lines (the correct number is 326037 for max_views=2 and 500M lines data file):")
!wc -l $OUTPUT_FOLDER/"bound_number_of_views_per_user.txt"

In [ ]:
#@title Hint
#@markdown Search for `combiners.Sample` and `beam.FlatMap`

In [ ]:
#@title Solution
def bound_number_of_views_per_user(views,max_views=2):
    return (
        views 
        | beam.Map(lambda mv: (mv.user_id, mv))
        # user_id -> list of MovieView
        | combiners.Sample.FixedSizePerKey(max_views)
        | "Remove keys" >> beam.Values()
        | "Unnest lists" >> beam.FlatMap(lambda x: x))

run_pipeline(bound_number_of_views_per_user)

print("\nContent of the output file:")
!head -5 $OUTPUT_FOLDER/"bound_number_of_views_per_user.txt"
print("\nNumber of lines:")
!wc -l $OUTPUT_FOLDER/"bound_number_of_views_per_user.txt"

## Exercise 6
Write a pipeline which computes rating histogram per movie, i.e. 
it outputs `PCollection((movie_id, histogram_tuple))`, where `histogram_tuple` is 
the `5` elements tuple, which contains the number of views with corresponding rating.

This exercise is for learning how to create custom combiners (look for `beam.CombineFn`).

**Note**: this exercise is a bit more difficult than previous ones.

In [ ]:
def movie_rating_histogram(views):
    # TODO: write your code here

run_pipeline(movie_rating_histogram)

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"movie_rating_histogram.txt"

In [ ]:
#@title Hint

#@markdown Have a look at `beam.CombineFn` `PTransform`, and `CombinePerKey`

In [ ]:
#@title Solution
class SumTuplesCombiner(beam.CombineFn):
    def __init__(self, ndim):
        self.ndim = ndim

    def create_accumulator(self):
        return [0] * self.ndim

    def add_input(self, accumulator, element):
        for i, b in enumerate(element):
            accumulator[i] += b
        return accumulator

    def merge_accumulators(self, accumulators):
        res = self.create_accumulator()
        for a in accumulators:
            self.add_input(res, a)
        return res

    def extract_output(self, accumulator):
        return accumulator


def movie_rating_histogram(views):
    def one_hot(rating):
        return (rating == 1, rating == 2, rating == 3, rating == 4, rating == 5)

    return (
        views 
        | beam.Map(lambda mv: (mv.movie_id, one_hot(mv.rating)))
        | beam.CombinePerKey(SumTuplesCombiner(ndim=5)))

run_pipeline(movie_rating_histogram)

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"movie_rating_histogram.txt"

## Exercise 7

This exercise has **2 parts**, the first one is about reading from files, the second is about using `join`.

In this exercise we will work with `MovieTitle`. The next code cell contains data class definition and code for parsing `MovieTitle` from string.

### Exercise 7a

Read and parse a movie titles file and output movie **names**. It should output `PCollection(str)`

In [ ]:
def read_movie_names(movie_titles):
  # TODO: write your code here

run_pipeline(read_movie_names, 
             pcollection_fn=netflix_movie_title_collection, 
             data_file=str(DATASET_FOLDER/"movie_titles.csv"))

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"read_movie_names.txt"

In [ ]:
#@title Hint

#@markdown Use the `netflix_movie_titles_collection` function from the `movie_titles.csv` data file.

In [ ]:
#@title Solution
def read_movie_names(movie_titles):
    return (
        movie_titles
        | "Extract movie names">> beam.Map(lambda title: title.name))

run_pipeline(read_movie_names, 
             pcollection_fn=netflix_movie_title_collection, 
             data_file=str(DATASET_FOLDER/"movie_titles.csv"))

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"read_movie_names.txt"

### Exercise 7b

This part of the exercise is for learning of using `join` (which is called `CoGroupByKey` in **Apache Beam**).

Output `PCollection((movie_name, number of views))`, only for movies with views.

Hint: reuse the function for number of views per movie from Exercise 3

In [ ]:
def movie_name_count_views(movie):
  # TODO: 1. reading movie titles similar to 7a

  # TODO: 2. compute movie_id_counts

  # TODO: 3. join 1 and 2 (search for CoGroupByKey)

run_pipeline(movie_name_count_views)

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"movie_name_count_views.txt"

In [ ]:
#@title Solution
def movie_name_count_views(views):
    movie_views_count = views_count_per_movie(views)  # PCollection[[Id, Count]]
    
    pipeline = views.pipeline  # re-using the same pipeline
    movie_title_ids = ( 
        netflix_movie_title_collection(pipeline) 
        | "To (movie_id, title)" >> beam.Map(lambda t: (t.movie_id, t))
    )

    # Join movie_views_count and movie_title_ids
    def process_merge(merge_dict):
        # Element Format: (movie_id, {"counts": [...], "titles": [...]})
        movie_id, join_info = merge_dict
        
        if not join_info:
            return # Skip
        
        if not len(join_info["counts"]) or not len(join_info["titles"]): 
            return [] # no views of this movie. It happens because only part of the views dataset is processed.
        
        yield (join_info["titles"][0].name, join_info["counts"][0])

    return (
        ({"counts": movie_views_count, "titles": movie_title_ids}) 
        | "Merge" >> beam.CoGroupByKey()
        | beam.ParDo(process_merge))

run_pipeline(movie_name_count_views)

print("\nContent of the output file:")
!head -10 $OUTPUT_FOLDER/"movie_name_count_views.txt"

## What's Next

The next [colab](https://github.com/OpenMined/PipelineDP/blob/main/docs/tutorial_1/2_beam_laplace_mechansim.ipynb) introduces Laplace mechanism and how to use it.